<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/FL-PSO-SGD/FLPSO_SGD_Clients_unbalanced_pre_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FLPSO-ADAM : desbalanceado, cada cliente tem 2 dígitos somente. Foram feitas mudanças para que o PSO atualizasse diretamente os pesos do modelo, mas não surtiu efeito.

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
from collections import OrderedDict

In [11]:
class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=256, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.to(device)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achatar o tensor de entrada
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc3(y)

        return y

In [12]:
# Definições dos hiperparâmetros
NUM_CLIENTES = 5
NUM_PARTICULAS = 20
NUM_RODADAS = 10
NUM_DIGITOS = 10
#INERCIA, C1, C2 = 0.8, 0.2, 0.2
INERCIA, C1, C2 = 0.8, 1.5, 1.0
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

# Criando o modelo global
modelo_global = MLP(DEVICE)
criterio = nn.CrossEntropyLoss()

training on cuda


In [13]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

class Particula:
    def __init__(self, particle_id, modelo_cliente):
        self.particle_id = particle_id
        self.device = modelo_cliente.device
        self.pesos = copy.deepcopy(modelo_cliente.state_dict())

        # Adiciona ruído leve nos pesos para quebrar simetria inicial
        for name in self.pesos:
            self.pesos[name] += 0.01 * torch.randn_like(self.pesos[name])
            #self.pesos[name] += 0.001 * torch.randn_like(self.pesos[name])

        self.melhor_pesos = copy.deepcopy(self.pesos)
        self.melhor_erro = float('inf')
        self.velocidade = {name: torch.zeros_like(param) for name, param in self.pesos.items()}

    def atualizar_pso(self, global_best_pesos, INERCIA, C1, C2):
        MAX_VELOCITY = 0.1  # Limite para evitar oscilações grandes
        for name in self.pesos:
            local_rand = random.random()
            global_rand = random.random()
            self.velocidade[name] = (
                INERCIA * self.velocidade[name]
                + C1 * local_rand * (self.melhor_pesos[name] - self.pesos[name])
                + C2 * global_rand * (global_best_pesos[name] - self.pesos[name])
            )

            # Clipping da velocidade
            self.velocidade[name] = torch.clamp(self.velocidade[name], -MAX_VELOCITY, MAX_VELOCITY)

            # Atualiza os pesos com a nova velocidade
            self.pesos[name] += self.velocidade[name]

    def avaliar_perda(self, modelo_cliente, criterio, dados):
        modelo_cliente.load_state_dict(self.pesos)
        modelo_cliente.eval()
        total_loss = 0

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = modelo_cliente(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)


class Cliente:
    def __init__(self, cliente_id, modelo_global, dados, test, num_particulas):
        self.cliente_id = cliente_id
        self.modelo = copy.deepcopy(modelo_global)  # Cada cliente tem seu próprio modelo
        self.dados = dados
        self.test = test
        self.num_particulas = num_particulas
        self.particulas = []
        self.melhor_particula = None
        self.inicializar_particulas(num_particulas)

    def inicializar_particulas(self, num_particulas):
        """Cria um conjunto de partículas associadas ao cliente."""
        self.particulas = [Particula(i, self.modelo) for i in range(num_particulas)]

    def treinar_com_pso(self, INERCIA, C1, C2, global_best_pesos, criterio):
        """Treina as partículas usando PSO e atualiza a melhor partícula local."""

        for particula in self.particulas:

            particula.atualizar_pso(global_best_pesos, INERCIA, C1, C2)
            erro = particula.avaliar_perda(self.modelo, criterio, self.test)
            if erro < particula.melhor_erro:
                particula.melhor_erro = erro
                particula.melhor_pesos = copy.deepcopy(particula.pesos)

        self.selecionar_melhor_particula()
        # modelo_global.load_state_dict(self.melhor_particula.pesos)

    def refinar_com_adam(self, criterio):
        """Refina os pesos da melhor partícula usando Adam."""
        self.modelo.load_state_dict(self.melhor_particula.melhor_pesos)
        # self.modelo.load_state_dict(modelo_global.state_dict())
        otimizador = optim.Adam(self.modelo.parameters(), lr=0.005, weight_decay=1e-5)

        self.modelo.train()
        for i in range(1):  # 5 épocas de refinamento com Adam
          for inputs, labels in self.dados:
              inputs, labels = inputs.to(self.modelo.device), labels.to(self.modelo.device)
              otimizador.zero_grad()
              outputs = self.modelo(inputs)
              loss = criterio(outputs, labels)
              loss.backward()
              otimizador.step()

        # Atualiza os pesos da melhor partícula com os pesos refinados pelo Adam
        self.melhor_particula.melhor_pesos = copy.deepcopy(self.modelo.state_dict())
        self.melhor_particula.melhor_erro = self.calcular_loss(self.modelo, criterio, self.test)

    def calcular_loss(self, modelo, criterio, dados):
        self.modelo.eval()
        total_loss = 0

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = modelo(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)

    def selecionar_melhor_particula(self):
        """Seleciona a melhor partícula do cliente."""
        self.melhor_particula = min(self.particulas, key=lambda p: p.melhor_erro)


def treinar_federado(modelo_global, clientes, criterio, num_rodadas, INERCIA, C1, C2, testloader):
    """Treina os clientes localmente e sincroniza com o servidor central, validando a acurácia."""

    melhor_peso_global = copy.deepcopy(modelo_global.state_dict())  # Inicializa com o modelo global
    melhor_erro_global = float('inf')
    soma = 0.0;

    for rodada in range(num_rodadas):
        resultados_rodada = []

        for cliente in clientes:
          if(rodada == 0):
            cliente_loss, cliente_accuracy = avaliar_modelo(cliente.modelo, criterio, testloader)
            print(f"cliente[{cliente.cliente_id}] loss {cliente_loss:.2f} accuracy {cliente_accuracy:.2f}%")
          cliente.treinar_com_pso(INERCIA, C1, C2, melhor_peso_global, criterio)  # Treino com PSO
          cliente.refinar_com_adam(criterio)  # Refinamento com Adam
          #erro_cliente = cliente.melhor_particula.melhor_erro  # Obtém o melhor erro do cliente
          pesos_cliente = cliente.melhor_particula.melhor_pesos  # Obtém os pesos do modelo do cliente
          resultados_rodada.append((pesos_cliente))
          #resultados_rodada.append((cliente.cliente_id, erro_cliente))

          media_pesos = average_state_dict(resultados_rodada)  # Média dos pesos dos clientes

          modelo_global.load_state_dict(media_pesos)  # Atualiza o modelo global com a média dos pesos

        #modelo_global.load_state_dict(melhor_peso_global)

        test_loss, test_accuracy = avaliar_modelo(modelo_global, criterio, testloader)
        soma += test_accuracy

        # if (rodada+1) % 10 == 0:
        print(f"Rodada {rodada+1}/{num_rodadas}:)")
        # print(f"Erro Global Atualizado: {melhor_erro_global:.4f}")
        print(f"Teste -> Perda: {test_loss:.4f}, Acurácia: {test_accuracy:.2f}%\n")

    print(f"Acurácia média: {soma/num_rodadas: .2f}")
    print("Treinamento Federado Finalizado!")

def avaliar_modelo(modelo, criterio, testloader):
    """Avalia o modelo global no conjunto de teste."""
    modelo.eval()  # Modo de avaliação
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples) * 100

    return test_loss, test_accuracy

def average_state_dict(state_dicts):
    # Cria um OrderedDict para armazenar a soma dos pesos
    avg_state_dict = OrderedDict()
    # Inicializa os tensores com zeros com o mesmo shape dos pesos do primeiro state_dict
    for key in state_dicts[0]:
        avg_state_dict[key] = torch.zeros_like(state_dicts[0][key])

    # Soma os pesos de cada state_dict
    for state_dict in state_dicts:
        for key in state_dict:
            avg_state_dict[key] += state_dict[key]

    # Divide cada tensor pela quantidade de state_dicts para obter a média
    for key in avg_state_dict:
        avg_state_dict[key] /= len(state_dicts)

    return avg_state_dict

def divide_trainset_and_testset_for_pre_training(trainset, testset):
  total_size = len(trainset )
  train_size = int(0.95 * total_size)
  pre_train_size =int(0.05 * total_size)

  total_size_test = len(testset)
  test_size = int(0.95 * total_size_test)
  pre_test_size = int(0.05 * total_size_test)

  train_subset, pre_train_subset = random_split(trainset, [train_size, pre_train_size])
  train_data = trainset.tensors[0][train_subset.indices]
  train_labels = trainset.tensors[1][train_subset.indices]
  pre_train_data = trainset.tensors[0][pre_train_subset.indices]
  pre_train_labels = trainset.tensors[1][pre_train_subset.indices]
  test_subset, pre_test_subset = random_split(testset, [test_size, pre_test_size])
  test_data = testset.tensors[0][test_subset.indices]
  test_labels = testset.tensors[1][test_subset.indices]
  pre_test_data = testset.tensors[0][pre_test_subset.indices]
  pre_test_labels = testset.tensors[1][pre_test_subset.indices]

  # Cria novos TensorDatasets (não Subsets)
  train_subset = TensorDataset(train_data, train_labels)
  pre_train_subset = TensorDataset(pre_train_data, pre_train_labels)
  test_subset = TensorDataset(test_data, test_labels)
  pre_test_subset = TensorDataset(pre_test_data, pre_test_labels)
  return train_subset, pre_train_subset, test_subset, pre_test_subset

def pre_train_global_model(client_pre_training, pre_test_loader, pre_train_loader, criterio, optimizer, num_epochs):

  client_pre_training.treinar_com_pso(INERCIA, C1, C2, client_pre_training.modelo.state_dict(), criterio)
  client_pre_training.refinar_com_adam(criterio)
  client_pre_training.modelo.load_state_dict(client_pre_training.melhor_particula.melhor_pesos)
  test_loss, test_accuracy = avaliar_modelo(client_pre_training.modelo, criterio, pre_test_loader)

  print(f"Pre training results: Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")

mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten
y_test = mnist_test.targets.numpy()

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten
y_train = mnist_train.targets.numpy()

# Normalização (como foi feito com o Iris)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)

trainset, pre_trainset, testset, pre_testset = divide_trainset_and_testset_for_pre_training(trainset, testset)
pre_trainset_loader = DataLoader(pre_trainset, batch_size=64, shuffle=True)
pre_testset_loader = DataLoader(pre_testset, batch_size=64, shuffle=False)

client_digit_mapping = {
    0: [0, 1],
    1: [2, 3],
    2: [4, 5],
    3: [6, 7],
    4: [8, 9]
}



def create_subset_two_digits(dataset, client_digit_mapping):
    # Extrai os labels do dataset
    targets = dataset.tensors[1].numpy()
    subsets = []
    for client_id, digits in client_digit_mapping.items():
        # Seleciona os índices cujos labels estejam na lista "digits"
        indices = np.where(np.isin(targets, digits))[0]
        np.random.shuffle(indices)  # Embaralha para garantir aleatoriedade
        subsets.append(Subset(dataset, indices))
    return subsets

train_subsets = create_subset_two_digits(trainset, client_digit_mapping)

# Criar DataLoaders para cada subset (cliente)
trainloaders = [DataLoader(train_subsets[i], batch_size=240, shuffle=True) for i in range(NUM_CLIENTES)]

testloader = DataLoader(testset, batch_size=64, shuffle=False)
client_pre_training = Cliente(-1, modelo_global, pre_trainset_loader, pre_testset_loader, 20)
pre_train_global_model(client_pre_training, pre_testset_loader, pre_trainset_loader, criterio, optim.Adam(modelo_global.parameters(), lr=0.001), 5)

for i, subset in enumerate(train_subsets):
    print("Cliente {} tem {} amostras (dígitos {}).".format(i, len(subset), client_digit_mapping[i]))

# Criando os clientes
clientes = [Cliente(i, client_pre_training.modelo, trainloaders[i], testloader, NUM_PARTICULAS) for i in range(NUM_CLIENTES)]

# Executando o treinamento federado
treinar_federado(modelo_global, clientes, criterio, NUM_RODADAS, INERCIA, C1, C2, testloader)

Pre training results: Loss: 0.4710, Accuracy: 87.20%
Cliente 0 tem 12029 amostras (dígitos [0, 1]).
Cliente 1 tem 11501 amostras (dígitos [2, 3]).
Cliente 2 tem 10690 amostras (dígitos [4, 5]).
Cliente 3 tem 11566 amostras (dígitos [6, 7]).
Cliente 4 tem 11214 amostras (dígitos [8, 9]).
cliente[0] loss 0.41 accuracy 88.28%
cliente[1] loss 0.41 accuracy 88.28%
cliente[2] loss 0.41 accuracy 88.28%
cliente[3] loss 0.41 accuracy 88.28%
cliente[4] loss 0.41 accuracy 88.28%
Rodada 1/10:)
Teste -> Perda: 0.6169, Acurácia: 89.28%

Rodada 2/10:)
Teste -> Perda: 0.6053, Acurácia: 86.62%

Rodada 3/10:)
Teste -> Perda: 0.5659, Acurácia: 88.96%

Rodada 4/10:)
Teste -> Perda: 0.5755, Acurácia: 89.68%

Rodada 5/10:)
Teste -> Perda: 0.7867, Acurácia: 83.05%

Rodada 6/10:)
Teste -> Perda: 0.7308, Acurácia: 84.85%

Rodada 7/10:)
Teste -> Perda: 0.6215, Acurácia: 82.46%

Rodada 8/10:)
Teste -> Perda: 0.7032, Acurácia: 87.55%

Rodada 9/10:)
Teste -> Perda: 0.7289, Acurácia: 83.62%

Rodada 10/10:)
Teste ->